In [1]:
from crbr.models.models import UNetApollo
import os
from clearml import OutputModel, Task
import numpy as np


raw_weights_fname = 'mlops-weights-128x128x128-seModel-hemorrhages-all-all-v18b-seModel_refine_v8.27-cycle-0-20.h5'
new_weights_name = "mlops-weights-128x128x128-seModel-hemorrhages-all-all-v18b-seModel_refine_v8.27-cycle-0-20_MODIFIED.h5"
raw_weights_path = os.path.join("/home/cerebriu/data/DM/MyCerebriu/Apollo/", raw_weights_fname)
modified_weights_path = os.path.join("/home/cerebriu/data/DM/MyCerebriu/Apollo/", new_weights_name)

2024-03-22 11:42:09.723940: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-22 11:42:09.757252: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 11:42:09.757281: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 11:42:09.757303: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-22 11:42:09.763325: I tensorflow/core/platform/cpu_feature_g

In [2]:
def get_layers_info(model_in):
    for i, layer in enumerate(model_in.layers):
        # Get the configuration of the current layer
        config = layer.get_config()

        # Print layer details
        print(f"Layer {i} | Name: {config.get('name')} | Type: {layer.__class__.__name__}")
        print(f"Config: {config}")

        # If the layer has weights, print their shapes
        if layer.weights:
            weights_shapes = [w.shape.as_list() for w in layer.weights]
            print(f"Weight shapes: {weights_shapes}")

### Create Original Apollo without deep supervision

In [3]:
# Load original Apollo architecture with NO supervision and load pretrained weights
model_pretrained = UNetApollo(
    n_classes = 5,
    n_channels = 3,
    deep_supervision=False
    )   
model_pretrained.build([None, 128, 128, 128, 3])
model_pretrained.load_weights(raw_weights_path, by_name=True)

### Create desired model architecture with ranodmly initialized weights

In [4]:
model_cmb = UNetApollo(
    n_classes = 2,
    n_channels = 1,
    deep_supervision=False
    )
model_cmb.build([None, 128, 128, 128, 1])


In [5]:
# get_layers_info(model)

In [6]:
model_pretrained.summary()

Model: "UNetApollo3D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 128, 128, 128,    0         
                             3)]                                 
                                                                 
 Encoder (Encoder)           multiple                  19138082  
                                                                 
 Decoder (Decoder)           multiple                  17165694  
                                                                 
 output_conv (Conv3D)        multiple                  165       
                                                                 
 SelfAttentionBlock (SelfAt  multiple                  18        
 tentionBlock)                                                   
                                                                 
Total params: 36303959 (138.49 MB)
Trainable params: 3

In [7]:
# get_layers_info(model_cmb)

In [8]:
# model_cmb.summary()

In [9]:

for i, (layer_old, layer_new) in enumerate(zip(model_pretrained.layers, model_cmb.layers)):
    # Print layer details
    print(f"OLD: Layer {i} | Name: {layer_old.name} | Type: {layer_old.__class__.__name__}")
    print(f"NEW: Layer {i} | Name: {layer_new.name} | Type: {layer_new.__class__.__name__}")

    if layer_old.weights:
        print(f"\tLayer has weights")
        weights = layer_old.get_weights()  # Get current weights
        new_weights = layer_new.get_weights()
        for w_old, w_new in zip(weights, new_weights):
            if w_old.shape != w_new.shape:
                print(f"\tDIFFERENCE FOUND --> Shape old: {w_old.shape}, shape new: {w_new.shape}")


    print("--------------------------------------------------\n")

# # Note: After performing these operations, the model's architecture has been altered, 
# # and it might not work as originally designed. Ensure to validate the model's performance.


OLD: Layer 0 | Name: input | Type: InputLayer
NEW: Layer 0 | Name: input | Type: InputLayer
--------------------------------------------------

OLD: Layer 1 | Name: Encoder | Type: Encoder
NEW: Layer 1 | Name: Encoder | Type: Encoder
	Layer has weights
	DIFFERENCE FOUND --> Shape old: (3, 3, 3, 3, 32), shape new: (3, 3, 3, 1, 32)
--------------------------------------------------

OLD: Layer 2 | Name: Decoder | Type: Decoder
NEW: Layer 2 | Name: Decoder | Type: Decoder
	Layer has weights
--------------------------------------------------

OLD: Layer 3 | Name: output_conv | Type: Conv3D
NEW: Layer 3 | Name: output_conv | Type: Conv3D
	Layer has weights
	DIFFERENCE FOUND --> Shape old: (1, 1, 1, 32, 5), shape new: (1, 1, 1, 32, 2)
	DIFFERENCE FOUND --> Shape old: (5,), shape new: (2,)
--------------------------------------------------

OLD: Layer 4 | Name: SelfAttentionBlock | Type: SelfAttentionBlock
NEW: Layer 4 | Name: SelfAttentionBlock | Type: SelfAttentionBlock
	Layer has weights
	

## Overwrite weights in random initialization with weights in Apollo prtrained
Changing dimensiosn when needed and keeping only weights of interest

In [10]:
for i, (layer_old, layer_new) in enumerate(zip(model_pretrained.layers, model_cmb.layers)):
    # Print layer details
    print(f"OLD: Layer {i} | Name: {layer_old.name} | Type: {layer_old.__class__.__name__}")
    print(f"NEW: Layer {i} | Name: {layer_new.name} | Type: {layer_new.__class__.__name__}")
    new_weights = []
    if layer_old.weights:
        weights = layer_old.get_weights()  # Get current weights
        weights_new = layer_new.get_weights()

        if layer_old.name == "SelfAttentionBlock":
            for w in weights:
                if w.shape == (3, 3):
                    # Keep only MIDDLE SWI/T2S
                    # Use slicing to keep the dimensions: [last_row:last_row+1, last_column:last_column+1]
                    modified_w = w[1:2, 1:2]  # This will have a shape of (1, 1)
                    print(f"\tExtracted weights of interest, from {w.shape} --> {modified_w.shape}")
                    new_weights.append(modified_w)
                else:
                    new_weights.append(w)  # For weights that don't match the criteria, keep them unchanged

        elif layer_old.name == "output_conv":
            for i, w in enumerate(weights):
                if w.shape == (1, 1, 1, 32, 5): 
                    modified_w = w[:,:,:,:, [0, 3]] # Keep only the weights for background and hemorrhages TODO: make sure it's this channel
                    print(f"\tExtracted weights of interest, from {w.shape} --> {modified_w.shape}")
                    new_weights.append(modified_w)
                elif w.shape == (5,):  # Bias adjustment
                    modified_w = w[[0, 3]]  # Get biases for background and hemo
                    print(f"\tExtracted weights of interest, from {w.shape} --> {modified_w.shape}")
                    new_weights.append(modified_w)
                else:
                    new_weights.append(w)

        elif layer_old.name == "Encoder":
            for i, w in enumerate(weights):
                if w.shape == (3, 3, 3, 3, 32): 
                    modified_w = w[:, :, :, 1:2, :] # Keep only the weights for T2S/SWI, shape new: (3, 3, 3, 1, 32)
                    print(f"\tExtracted weights of interest, from {w.shape} --> {modified_w.shape}")
                    new_weights.append(modified_w)
                else:
                    new_weights.append(w)
        else:
            for i, w in enumerate(weights):
                new_weights.append(w)

    # Set the modified weights back to the layer, if any modification was made
    if len(new_weights)>0:
        layer_new.set_weights(new_weights)
        print("\tWeights have been overwritten with pretrained in this layer")
        if layer_old.name in ["SelfAttentionBlock"
                            #   , "output_conv" 
                            ]:
            print()
            print()
            print()
            print("-- Apollo weights --")
            print(weights)
            print("-- Default initialized --")
            print(weights_new)
            print("-- Apollo pretrained --")
            print(new_weights)

    print("--------------------------------------------------\n")

# # Note: After performing these operations, the model's architecture has been altered, 
# # and it might not work as originally designed. Ensure to validate the model's performance.


OLD: Layer 0 | Name: input | Type: InputLayer
NEW: Layer 0 | Name: input | Type: InputLayer
--------------------------------------------------

OLD: Layer 1 | Name: Encoder | Type: Encoder
NEW: Layer 1 | Name: Encoder | Type: Encoder
	Extracted weights of interest, from (3, 3, 3, 3, 32) --> (3, 3, 3, 1, 32)
	Weights have been overwritten with pretrained in this layer
--------------------------------------------------

OLD: Layer 2 | Name: Decoder | Type: Decoder
NEW: Layer 2 | Name: Decoder | Type: Decoder
	Weights have been overwritten with pretrained in this layer
--------------------------------------------------

OLD: Layer 3 | Name: output_conv | Type: Conv3D
NEW: Layer 3 | Name: output_conv | Type: Conv3D
	Extracted weights of interest, from (1, 1, 1, 32, 5) --> (1, 1, 1, 32, 2)
	Extracted weights of interest, from (5,) --> (2,)
	Weights have been overwritten with pretrained in this layer
--------------------------------------------------

OLD: Layer 4 | Name: SelfAttentionBlock 

In [11]:
model_cmb.get_layer(
    "SelfAttentionBlock"
).get_weights()

# Output should look like this:
# -- Apollo pretrained --
# [array([[-0.7917804]], dtype=float32), array([[-0.33530304]], dtype=flo   at32)]

[array([[-0.7917804]], dtype=float32), array([[-0.33530304]], dtype=float32)]

In [12]:
model_cmb.save_weights(modified_weights_path)